In [79]:
import pandas as pd
import os
from pyspark.sql import SparkSession

# -------------------------
# 1️⃣ RUTAS DE ARCHIVOS
# -------------------------
base = "/home/asa117/Descargas/zzz_ficheros_laliga"
players_csv       = os.path.join(base, "players.csv")
players_stats_csv = os.path.join(base, "players_stats.csv")
season_csv        = os.path.join(base, "season-2425.csv")
clas_csv          = os.path.join(base, "clasification.csv")

# -------------------------
# 2️⃣ CARGAR CSVs
# -------------------------
players_pd       = pd.read_csv(players_csv, encoding="latin1", sep=",", engine="python")
players_stats_pd = pd.read_csv(players_stats_csv, encoding="latin1", sep=",", engine="python")
season_pd        = pd.read_csv(season_csv, encoding="latin1", sep=",", engine="python")
clas_pd          = pd.read_csv(clas_csv, encoding="utf-8-sig", sep=",", engine="python")  # BOM eliminado

# Normalizar nombres de columnas
players_pd.columns = players_pd.columns.str.strip()
players_stats_pd.columns = players_stats_pd.columns.str.strip()
season_pd.columns = season_pd.columns.str.strip()
clas_pd.columns = clas_pd.columns.str.strip()

# -------------------------
# 3️⃣ LIMPIEZA AGE
# -------------------------
def fix_age(age_val):
    try:
        age = int(str(age_val).split("-")[0])
        return age
    except:
        return pd.NA

players_pd['Age'] = players_pd['Age'].apply(fix_age)

# -------------------------
# 4️⃣ MERGE CON players_stats.csv (Goals y Assists correctos)
# -------------------------
players_stats_pd.rename(columns={'name':'Player', 'goals':'Goals', 'assists':'Assists'}, inplace=True)

players_pd = players_pd[['Player','Team','Position','Age']].merge(
    players_stats_pd[['Player','Goals','Assists']],
    on='Player',
    how='left'
)

players_pd['Goals'] = players_pd['Goals'].fillna(0).astype('Int64')
players_pd['Assists'] = players_pd['Assists'].fillna(0).astype('Int64')

players_pd = players_pd.drop_duplicates(subset=['Player','Team'])

# -------------------------
# 5️⃣ LIMPIEZA SEASON
# -------------------------
num_cols_season = ['FTHG','FTAG','HTHG','HTAG','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']
for c in num_cols_season:
    if c in season_pd.columns:
        season_pd[c] = pd.to_numeric(season_pd[c], errors='coerce').astype('Int64')

season_pd['Date'] = pd.to_datetime(season_pd['Date'], dayfirst=True, errors='coerce')

# -------------------------
# 6️⃣ LIMPIEZA CLASIFICATION
# -------------------------
def split_goals(x):
    try:
        gf, ga = str(x).split(":")
        return int(gf), int(ga)
    except:
        return pd.NA, pd.NA

gf_ga = clas_pd['Goals'].apply(split_goals)
clas_pd['GoalsFor'] = gf_ga.apply(lambda t: t[0])
clas_pd['GoalsAgainst'] = gf_ga.apply(lambda t: t[1])

keep_cols_clas = ['Position','Club','PJ','G','E','P','GoalsFor','GoalsAgainst','Ptos']
clas_pd = clas_pd[keep_cols_clas]

for c in ['Position','PJ','G','E','P','GoalsFor','GoalsAgainst','Ptos']:
    clas_pd[c] = pd.to_numeric(clas_pd[c], errors='coerce').astype('Int64')

# -------------------------
# 7️⃣ CREAR SPARK DATAFRAMES
# -------------------------
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LaLiga 24-25 Analysis") \
    .getOrCreate()

spark_players = spark.createDataFrame(players_pd)
spark_season  = spark.createDataFrame(season_pd)
spark_clas    = spark.createDataFrame(clas_pd)

# -------------------------
# 8️⃣ GUARDAR TABLAS HIVE-LIKE
# -------------------------
warehouse_dir = os.path.join(base, "spark-warehouse")
os.makedirs(warehouse_dir, exist_ok=True)
spark.sql(f"CREATE DATABASE IF NOT EXISTS laliga LOCATION '{warehouse_dir}'")

spark_players.write.mode("overwrite").saveAsTable("laliga.players")
spark_season.write.mode("overwrite").saveAsTable("laliga.season_2425")
spark_clas.write.mode("overwrite").saveAsTable("laliga.clasification")

spark.sql("SHOW TABLES IN laliga").show(truncate=False)

print("✅ Datos limpios, Spark DataFrames creados y tablas guardadas")


/tmp/ipykernel_43559/1371013021.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  season_pd['Date'] = pd.to_datetime(season_pd['Date'], dayfirst=True, errors='coerce')


+---------+-------------+-----------+
|namespace|tableName    |isTemporary|
+---------+-------------+-----------+
|laliga   |clasification|false      |
|laliga   |players      |false      |
|laliga   |season_2425  |false      |
|         |clas_view    |true       |
|         |players_view |true       |
|         |players_view2|true       |
+---------+-------------+-----------+

✅ Datos limpios, Spark DataFrames creados y tablas guardadas


In [77]:
spark_players.createOrReplaceTempView("players_view")
top_scorers = spark.sql("""
    SELECT Player, Team, Goals, Assists
    FROM players_view
    ORDER BY Goals DESC
    LIMIT 10
""")
top_scorers.show(truncate=False)

spark_clas.createOrReplaceTempView("clas_view")
clas_diff = spark.sql("""
    SELECT Club, PJ, G, E, P, GoalsFor, GoalsAgainst,
           (GoalsFor - GoalsAgainst) AS GoalDifference, Ptos
    FROM clas_view
    ORDER BY Ptos DESC
""")
clas_diff.show(truncate=False)



+------------------+----------------+-----+-------+
|Player            |Team            |Goals|Assists|
+------------------+----------------+-----+-------+
|Kylian MbappÃ©    |Real Madrid     |31   |3      |
|Robert Lewandowski|Barcelona       |27   |2      |
|Ante Budimir      |Osasuna         |21   |4      |
|Alexander SÃ¸rloth|AtlÃ©tico Madrid|20   |2      |
|Ayoze PÃ©rez      |Villarreal      |19   |2      |
|Raphinha          |Barcelona       |18   |10     |
|JuliÃ¡n Ãlvarez  |AtlÃ©tico Madrid|17   |4      |
|Oihan Sancet      |Athletic Club   |15   |1      |
|Javi Puado        |Espanyol        |12   |4      |
|Hugo Duro         |Valencia        |11   |2      |
+------------------+----------------+-----+-------+

+---------------+---+---+---+---+--------+------------+--------------+----+
|Club           |PJ |G  |E  |P  |GoalsFor|GoalsAgainst|GoalDifference|Ptos|
+---------------+---+---+---+---+--------+------------+--------------+----+
|FC Barcelona   |38 |28 |4  |6  |102     |3

In [78]:
import pandas as pd
import os
from pyspark.sql import SparkSession

# =====================================================
# 1️⃣ RUTAS DE ARCHIVOS
# =====================================================
base = "/home/asa117/Descargas/zzz_ficheros_laliga"

players_csv       = os.path.join(base, "players.csv")
players_stats_csv = os.path.join(base, "players_stats.csv")
season_csv        = os.path.join(base, "season-2425.csv")
clas_csv          = os.path.join(base, "clasification.csv")

# =====================================================
# 2️⃣ CARGAR CSVs
# =====================================================
players_pd       = pd.read_csv(players_csv, encoding="latin1", sep=",", engine="python")
players_stats_pd = pd.read_csv(players_stats_csv, encoding="latin1", sep=",", engine="python")
season_pd        = pd.read_csv(season_csv, encoding="latin1", sep=",", engine="python")
clas_pd          = pd.read_csv(clas_csv, encoding="utf-8-sig", sep=",", engine="python")

# Normalizar columnas
for df in [players_pd, players_stats_pd, season_pd, clas_pd]:
    df.columns = df.columns.str.strip()

print("✅ CSVs cargados correctamente")

# =====================================================
# 3️⃣ LIMPIEZA COLUMNA AGE
# =====================================================
def fix_age(age_val):
    try:
        # Tomar solo la parte antes del guión
        age = int(str(age_val).split("-")[0])
        return age
    except:
        return pd.NA

players_pd["Age"] = players_pd["Age"].apply(fix_age)

# =====================================================
# 4️⃣ MERGE CON players_stats.csv (GOALS y ASSISTS correctos)
# =====================================================
players_stats_pd.rename(
    columns={"name": "Player", "goals": "Goals", "assists": "Assists"},
    inplace=True
)

# Reducir a columnas útiles
players_pd = players_pd[["Player", "Team", "Position", "Age"]]

# Merge por nombre de jugador
merged_players = pd.merge(
    players_pd,
    players_stats_pd[["Player", "Goals", "Assists"]],
    on="Player",
    how="left"
)

# Limpiar tipos y duplicados
merged_players["Goals"] = merged_players["Goals"].fillna(0).astype("Int64")
merged_players["Assists"] = merged_players["Assists"].fillna(0).astype("Int64")
merged_players.drop_duplicates(subset=["Player", "Team"], inplace=True)

# =====================================================
# 5️⃣ LIMPIEZA SEASON (PARTIDOS)
# =====================================================
num_cols_season = [
    "FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST",
    "HF","AF","HC","AC","HY","AY","HR","AR"
]
for c in num_cols_season:
    if c in season_pd.columns:
        season_pd[c] = pd.to_numeric(season_pd[c], errors="coerce").astype("Int64")

season_pd["Date"] = pd.to_datetime(season_pd["Date"], dayfirst=True, errors="coerce")

# =====================================================
# 6️⃣ LIMPIEZA CLASIFICATION
# =====================================================
def split_goals(x):
    try:
        gf, ga = str(x).split(":")
        return int(gf), int(ga)
    except:
        return pd.NA, pd.NA

gf_ga = clas_pd["Goals"].apply(split_goals)
clas_pd["GoalsFor"] = gf_ga.apply(lambda t: t[0])
clas_pd["GoalsAgainst"] = gf_ga.apply(lambda t: t[1])

keep_cols_clas = ["Position", "Club", "PJ", "G", "E", "P", "GoalsFor", "GoalsAgainst", "Ptos"]
clas_pd = clas_pd[keep_cols_clas]

for c in ["Position", "PJ", "G", "E", "P", "GoalsFor", "GoalsAgainst", "Ptos"]:
    clas_pd[c] = pd.to_numeric(clas_pd[c], errors="coerce").astype("Int64")

# =====================================================
# 7️⃣ CREAR SESIÓN SPARK
# =====================================================
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LaLiga 24-25 Analysis") \
    .config("spark.sql.warehouse.dir", os.path.join(base, "spark-warehouse")) \
    .getOrCreate()

spark_players = spark.createDataFrame(merged_players)
spark_season  = spark.createDataFrame(season_pd)
spark_clas    = spark.createDataFrame(clas_pd)

print("✅ DataFrames Spark creados correctamente")

# =====================================================
# 8️⃣ EXPORTAR TABLAS LIMPIAS A CSV (para Power BI)
# =====================================================
output_dir = os.path.join(base, "clean_data")
os.makedirs(output_dir, exist_ok=True)

spark_players.toPandas().to_csv(os.path.join(output_dir, "players_clean.csv"), index=False)
spark_season.toPandas().to_csv(os.path.join(output_dir, "season_clean.csv"), index=False)
spark_clas.toPandas().to_csv(os.path.join(output_dir, "clasification_clean.csv"), index=False)

print(f"✅ Datos exportados a: {output_dir}")
print("Ahora puedes copiar esos CSV a Windows y cargarlos en Power BI.")


✅ CSVs cargados correctamente


/tmp/ipykernel_43559/2455910173.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  season_pd["Date"] = pd.to_datetime(season_pd["Date"], dayfirst=True, errors="coerce")
/tmp/ipykernel_43559/2455910173.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clas_pd[c] = pd.to_numeric(clas_pd[c], errors="coerce").astype("Int64")
/tmp/ipykernel_43559/2455910173.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

✅ DataFrames Spark creados correctamente
✅ Datos exportados a: /home/asa117/Descargas/zzz_ficheros_laliga/clean_data
Ahora puedes copiar esos CSV a Windows y cargarlos en Power BI.


In [57]:
from pyspark.sql import functions as F

# =========================================================
# 1️⃣ Promedio de goles por club (Goles marcados / 38 jornadas)
# =========================================================
avg_goals = spark.sql("""
SELECT 
    Club,
    ROUND(SUM(GoalsFor) / 38, 2) AS Promedio_Goles
FROM laliga.clasification
GROUP BY Club
ORDER BY Promedio_Goles DESC
""")
print("⚽️ Promedio de goles por club (por jornada):")
avg_goals.show(10)

# =========================================================
# 2️⃣ Top 10 goleadores
# =========================================================
top_scorers = spark.sql("""
SELECT 
    Player,
    Team,
    SUM(Goals) AS Goles
FROM laliga.players
GROUP BY Player, Team
HAVING Goles > 0
ORDER BY Goles DESC
LIMIT 10
""")
print("🏆 Top 10 goleadores de la temporada:")
top_scorers.show(10)

# =========================================================
# 3️⃣ Promedio de edad por equipo
# =========================================================
avg_age = spark.sql("""
SELECT 
    Team,
    ROUND(AVG(Age), 1) AS Edad_Promedio
FROM laliga.players
WHERE Age IS NOT NULL
GROUP BY Team
ORDER BY Edad_Promedio ASC
""")
print("👶 Promedio de edad por equipo:")
avg_age.show(10)

# =========================================================
# 4️⃣ Promedio de minutos por jugador
# =========================================================
avg_minutes = spark.sql("""
SELECT 
    Team,
    ROUND(AVG(Minutes), 0) AS Minutos_Promedio
FROM laliga.players
GROUP BY Team
ORDER BY Minutos_Promedio DESC
""")
print("⏱️ Promedio de minutos jugados por equipo:")
avg_minutes.show(10)

# =========================================================
# 5️⃣ Correlación entre minutos y goles
# =========================================================
players_df = spark.table("laliga.players")

corr = players_df.select(
    F.corr(F.col("Minutes"), F.col("Goals")).alias("Corr_Minutes_Goals")
).collect()[0]["Corr_Minutes_Goals"]

print(f"📈 Correlación entre minutos jugados y goles marcados: {corr:.3f}")

# =========================================================
# 6️⃣ Ranking combinado (goles + asistencias)
# =========================================================
combo = spark.sql("""
SELECT 
    Player,
    Team,
    SUM(Goals) AS Goles,
    SUM(Assists) AS Asistencias,
    (SUM(Goals) + SUM(Assists)) AS Participaciones_Totales
FROM laliga.players
GROUP BY Player, Team
ORDER BY Participaciones_Totales DESC
LIMIT 10
""")
print("🥇 Ranking combinado (goles + asistencias):")
combo.show(10)


⚽️ Promedio de goles por club (por jornada):
+-------------+--------------+
|         Club|Promedio_Goles|
+-------------+--------------+
| FC Barcelona|          2.68|
|  Real Madrid|          2.05|
|   Villarreal|          1.87|
|     Atlético|          1.79|
|     RC Celta|          1.55|
|   Real Betis|           1.5|
|Athletic Club|          1.42|
|   CA Osasuna|          1.26|
|  Valencia CF|          1.16|
|    Girona FC|          1.16|
+-------------+--------------+
only showing top 10 rows

🏆 Top 10 goleadores de la temporada:


+------------------+---------------+-----+
|            Player|           Team|Goles|
+------------------+---------------+-----+
|Robert Lewandowski|      Barcelona|   15|
|          Raphinha|      Barcelona|   11|
|      Ante Budimir|        Osasuna|    9|
|   Vinicius Júnior|    Real Madrid|    8|
|     Kylian Mbappé|    Real Madrid|    8|
|      Oihan Sancet|  Athletic Club|    7|
|       Ayoze Pérez|     Villarreal|    7|
|    Dodi Lukebakio|        Sevilla|    6|
| Alexander Sørloth|Atlético Madrid|    6|
|        Javi Puado|       Espanyol|    5|
+------------------+---------------+-----+

👶 Promedio de edad por equipo:
+-------------+-------------+
|         Team|Edad_Promedio|
+-------------+-------------+
|    Barcelona|         23.6|
|     Valencia|         24.1|
|Real Sociedad|         25.0|
|     Espanyol|         25.4|
|   Valladolid|         25.4|
|      Sevilla|         26.0|
|       Getafe|         26.1|
|   Celta Vigo|         26.2|
|       Alavés|         26.4|
|  Rea

[Stage 117:==============>                                          (1 + 3) / 4]

+------------------+---------------+-----+-----------+-----------------------+
|            Player|           Team|Goles|Asistencias|Participaciones_Totales|
+------------------+---------------+-----+-----------+-----------------------+
|          Raphinha|      Barcelona|   11|          6|                     17|
|Robert Lewandowski|      Barcelona|   15|          2|                     17|
|   Vinicius Júnior|    Real Madrid|    8|          5|                     13|
|      Lamine Yamal|      Barcelona|    5|          8|                     13|
|      Ante Budimir|        Osasuna|    9|          1|                     10|
| Antoine Griezmann|Atlético Madrid|    5|          4|                      9|
|     Kylian Mbappé|    Real Madrid|    8|          1|                      9|
|        Alex Baena|     Villarreal|    3|          5|                      8|
| Alexander Sørloth|Atlético Madrid|    6|          2|                      8|
|        Iago Aspas|     Celta Vigo|    5|          

In [31]:
num_cols_clas = ['Position','PJ','G','E','P','Diff','Ptos']
for c in num_cols_clas:
    if c in clas_pd.columns:
        clas_pd[c] = pd.to_numeric(clas_pd[c], errors='coerce').astype('Int64')
        print(f"✔ {c} sample:", clas_pd[c].head(3).tolist())



✔ PJ sample: [38, 38, 38]
✔ G sample: [28, 26, 22]
✔ E sample: [4, 6, 10]
✔ P sample: [6, 6, 6]
✔ Ptos sample: [88, 84, 76]


In [32]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LaLiga 24-25 Analysis") \
    .getOrCreate()

spark_players = spark.createDataFrame(players_pd)
spark_season  = spark.createDataFrame(season_pd)
spark_clas    = spark.createDataFrame(clas_pd)

print("✅ Spark DataFrames creados")
spark_players.printSchema()
spark_season.printSchema()
spark_clas.printSchema()


✅ Spark DataFrames creados
root
 |-- Player: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- #: long (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Minutes: long (nullable = true)
 |-- Goals: long (nullable = true)
 |-- Assists: long (nullable = true)
 |-- Penalty Shoot on Goal: long (nullable = true)
 |-- Penalty Shoot: long (nullable = true)
 |-- Total Shoot: long (nullable = true)
 |-- Shoot on Target: long (nullable = true)
 |-- Yellow Cards: long (nullable = true)
 |-- Red Cards: long (nullable = true)
 |-- Touches: long (nullable = true)
 |-- Dribbles: long (nullable = true)
 |-- Tackles: long (nullable = true)
 |-- Blocks: long (nullable = true)
 |-- Expected Goals (xG): double (nullable = true)
 |-- Non-Penalty xG (npxG): double (nullable = true)
 |-- Expected Assists (xAG): double (nullable = true)
 |-- Shot-Creating Actions: long (nullable = true)
 |-- Goal-Creating Acti

In [33]:
#pasamos a hive

warehouse_dir = os.path.join(base, "spark-warehouse")
os.makedirs(warehouse_dir, exist_ok=True)

spark.sql(f"CREATE DATABASE IF NOT EXISTS laliga LOCATION '{warehouse_dir}'")

spark_players.write.mode("overwrite").saveAsTable("laliga.players")
spark_season.write.mode("overwrite").saveAsTable("laliga.season_2425")
spark_clas.write.mode("overwrite").saveAsTable("laliga.clasification")

spark.sql("SHOW TABLES IN laliga").show(truncate=False)
print("✅ Tablas guardadas y listas para consultas SQL")


+---------+-------------+-----------+
|namespace|tableName    |isTemporary|
+---------+-------------+-----------+
|laliga   |clasification|false      |
|laliga   |players      |false      |
|laliga   |season_2425  |false      |
+---------+-------------+-----------+

✅ Tablas guardadas y listas para consultas SQL


In [34]:
# Promedio de goles por equipo según players.csv
avg_goals = spark.sql("""
SELECT Team, ROUND(AVG(Goals),2) AS avg_goals
FROM laliga.players
GROUP BY Team
ORDER BY avg_goals DESC
LIMIT 10
""")

avg_goals.show(truncate=False)



+----------------+---------+
|Team            |avg_goals|
+----------------+---------+
|Barcelona       |0.19     |
|Real Madrid     |0.14     |
|AtlÃ©tico Madrid|0.11     |
|Villarreal      |0.11     |
|Celta Vigo      |0.09     |
|Athletic Club   |0.09     |
|Girona          |0.09     |
|Osasuna         |0.08     |
|Las Palmas      |0.07     |
|AlavÃ©s         |0.07     |
+----------------+---------+



In [35]:
spark.sql("""
SELECT Club, SUM(GoalsFor) AS total_goals_for, SUM(GoalsAgainst) AS total_goals_against
FROM laliga.clasification
GROUP BY Club
ORDER BY total_goals_for DESC
""").show(truncate=False)

+---------------+---------------+-------------------+
|Club           |total_goals_for|total_goals_against|
+---------------+---------------+-------------------+
|FC Barcelona   |102            |39                 |
|Real Madrid    |78             |38                 |
|Villarreal     |71             |51                 |
|AtlÃ©tico      |68             |30                 |
|RC Celta       |59             |57                 |
|Real Betis     |57             |50                 |
|Athletic Club  |54             |29                 |
|CA Osasuna     |48             |52                 |
|Valencia CF    |44             |54                 |
|Girona FC      |44             |60                 |
|Sevilla FC     |42             |55                 |
|Rayo Vallecano |41             |45                 |
|RCD Espanyol   |40             |51                 |
|UD Las Palmas  |40             |61                 |
|CD LeganÃ©s    |39             |56                 |
|AlavÃ©s        |38         

In [36]:
spark.sql("""
SELECT Player, Team, Goals
FROM laliga.players
ORDER BY Goals DESC NULLS LAST
LIMIT 10
""").show(truncate=False)


+----------------------+-------------+-----+
|Player                |Team         |Goals|
+----------------------+-------------+-----+
|Raphinha              |Barcelona    |3    |
|Robert Lewandowski    |Barcelona    |3    |
|Javi Puado            |Espanyol     |3    |
|Vinicius JÃºnior      |Real Madrid  |3    |
|Ayoze PÃ©rez          |Villarreal   |2    |
|Kylian MbappÃ©        |Real Madrid  |2    |
|Orri Steinn Ãskarsson|Real Sociedad|2    |
|Robert Lewandowski    |Barcelona    |2    |
|Ante Budimir          |Osasuna      |2    |
|Lamine Yamal          |Barcelona    |2    |
+----------------------+-------------+-----+

